In [1]:
import cobra

In [2]:
import pandas as pd
from cobra.io import load_model

In [3]:
from cobra.flux_analysis import flux_variability_analysis

In [4]:
model = load_model("iJO1366") #no need to import from matlab

In [5]:
KOlist = ('GLCabcpp', 'GLCptspp', 'HEX1', 'PGI', 'PFK', 'FBA', 'TPI', 'GAPD', 'PGK', 'PGM', 'ENO', 'PYK', 'LDH_D', 'PFL', 'ALCD2x', 'PTAr', 'ACKr', 'G6PDH2r', 'PGL', 'GND', 'RPI', 'RPE', 'TKT1', 'TALA', 'TKT2', 'FUM', 'FRD2', 'SUCOAS', 'AKGDH', 'ACONTa', 'ACONTb', 'ICDHyr', 'CS', 'MDH',  'MDH2', 'MDH3', 'ACALD')

In [6]:
len(KOlist)

37

### WT

In [7]:
biomass = "BIOMASS_Ec_iJO1366_core_53p95M"

In [8]:
model.objective = biomass

In [9]:
solution = model.optimize()
print(solution)

<Solution 0.982 at 0x7effd84f2430>


In [10]:
model.reactions.get_by_id("EX_succ_e").summary().to_frame().loc["EX_succ_e", "flux"]

0.0

### MT

In [11]:
model.objective = 'EX_succ_e'

In [12]:
print(model.objective.expression)
print(model.objective.direction)

1.0*EX_succ_e - 1.0*EX_succ_e_reverse_a9039
max


In [13]:
solution = model.optimize()
print(solution)

<Solution 17.096 at 0x7effd82d2d60>


### MT 0.5 Growth rate

In [14]:
model.reactions.get_by_id(biomass).lower_bound = 0.98*0.5

In [15]:
solution = model.optimize()
print(solution)

<Solution 8.582 at 0x7effd8b4aac0>


In [16]:
flux_variability_analysis(model, "EX_succ_e", loopless=True)

,minimum,maximum
EX_succ_e,8.581712,8.581712


### Sim Comb

In [17]:
from itertools import combinations

In [26]:
#KOlist = ['GAPD', 'PGK', 'PGM', 'ENO', 'TPI', 'ACONTb', 'ACONTa', 'CS', 'FUM', 'MDH', 'GLCptspp', 'PGI']

targetNum = 12
targetList = KOlist[:targetNum]
combList = []
for i in range(1,targetNum+1):
    combList.extend(list(combinations(targetList, i)))

In [27]:
len(combList)

4095

In [28]:
model.objective = 'EX_succ_e'

In [29]:
objList = []
for comb in combList:
    with model:
        for rec in comb:
            model.reactions.get_by_id(rec).knock_out()
        objList.append(model.optimize().objective_value)
        #print(model.optimize().objective_value)

/home/chenjunyu/miniconda3/envs/cobra/lib/python3.8/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
/home/chenjunyu/miniconda3/envs/cobra/lib/python3.8/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
/home/chenjunyu/miniconda3/envs/cobra/lib/python3.8/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
/home/chenjunyu/miniconda3/envs/cobra/lib/python3.8/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
/home/chenjunyu/miniconda3/envs/cobra/lib/python3.8/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
/home/chenjunyu/miniconda3/envs/cobra/lib/python3.8/sit

In [ ]:
combList[:15]

In [ ]:
combList[15]

In [30]:
cList = []
for comb in combList:
    cList.append(comb[0])

In [57]:
df = pd.DataFrame()
df["comb"] = combList
df["obj"] = objList

In [32]:
df

,comb,obj
0,"(GLCabcpp,)",8.581712
1,"(GLCptspp,)",8.451703
2,"(HEX1,)",8.581712
3,"(PGI,)",8.471788
4,"(PFK,)",8.581712
...,...,...
4090,"(GLCabcpp, GLCptspp, HEX1, PFK, FBA, TPI, GAPD...",0.000000
4091,"(GLCabcpp, GLCptspp, PGI, PFK, FBA, TPI, GAPD,...",0.000000
4092,"(GLCabcpp, HEX1, PGI, PFK, FBA, TPI, GAPD, PGK...",0.000000
4093,"(GLCptspp, HEX1, PGI, PFK, FBA, TPI, GAPD, PGK...",0.000000


### add 20% random 

In [34]:
import random

In [39]:
j = random.uniform(0.8, 1.2)

0.9068176217276738

In [40]:
objRanList = []
for i in objList:
    if i == 0:
        objRanList.append(random.uniform(0, 2))
    else:
        j = random.uniform(0.8, 1.2)
        objRanList.append(i*j)

In [58]:
df["obj_rand"] = objRanList

In [59]:
df

,comb,obj,obj_rand
0,"(GLCabcpp,)",8.581712,8.446460
1,"(GLCptspp,)",8.451703,8.634571
2,"(HEX1,)",8.581712,10.190002
3,"(PGI,)",8.471788,8.425687
4,"(PFK,)",8.581712,10.222978
...,...,...,...
4090,"(GLCabcpp, GLCptspp, HEX1, PFK, FBA, TPI, GAPD...",0.000000,0.100671
4091,"(GLCabcpp, GLCptspp, PGI, PFK, FBA, TPI, GAPD,...",0.000000,0.813981
4092,"(GLCabcpp, HEX1, PGI, PFK, FBA, TPI, GAPD, PGK...",0.000000,0.720188
4093,"(GLCptspp, HEX1, PGI, PFK, FBA, TPI, GAPD, PGK...",0.000000,1.095238


In [43]:
df.to_csv("Ecoli_Suc_KO_12_rand.csv")

In [33]:
df.to_csv("Ecoli_Suc_KO_12.csv")

In [44]:
import numpy as np

In [60]:
for ko in KOlist[:12]:
    df[ko] = 0

In [61]:
df

,comb,obj,obj_rand,GLCabcpp,GLCptspp,HEX1,PGI,PFK,FBA,TPI,GAPD,PGK,PGM,ENO,PYK
0,"(GLCabcpp,)",8.581712,8.446460,0,0,0,0,0,0,0,0,0,0,0,0
1,"(GLCptspp,)",8.451703,8.634571,0,0,0,0,0,0,0,0,0,0,0,0
2,"(HEX1,)",8.581712,10.190002,0,0,0,0,0,0,0,0,0,0,0,0
3,"(PGI,)",8.471788,8.425687,0,0,0,0,0,0,0,0,0,0,0,0
4,"(PFK,)",8.581712,10.222978,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4090,"(GLCabcpp, GLCptspp, HEX1, PFK, FBA, TPI, GAPD...",0.000000,0.100671,0,0,0,0,0,0,0,0,0,0,0,0
4091,"(GLCabcpp, GLCptspp, PGI, PFK, FBA, TPI, GAPD,...",0.000000,0.813981,0,0,0,0,0,0,0,0,0,0,0,0
4092,"(GLCabcpp, HEX1, PGI, PFK, FBA, TPI, GAPD, PGK...",0.000000,0.720188,0,0,0,0,0,0,0,0,0,0,0,0
4093,"(GLCptspp, HEX1, PGI, PFK, FBA, TPI, GAPD, PGK...",0.000000,1.095238,0,0,0,0,0,0,0,0,0,0,0,0


In [62]:
len(df)

4095

In [63]:
df["comb"][10]

('ENO',)

In [64]:
for i in range(len(df)):
    #print(i)
    for j in df["comb"][i]:
        df.loc[i, j] = 1

In [65]:
df

,comb,obj,obj_rand,GLCabcpp,GLCptspp,HEX1,PGI,PFK,FBA,TPI,GAPD,PGK,PGM,ENO,PYK
0,"(GLCabcpp,)",8.581712,8.446460,1,0,0,0,0,0,0,0,0,0,0,0
1,"(GLCptspp,)",8.451703,8.634571,0,1,0,0,0,0,0,0,0,0,0,0
2,"(HEX1,)",8.581712,10.190002,0,0,1,0,0,0,0,0,0,0,0,0
3,"(PGI,)",8.471788,8.425687,0,0,0,1,0,0,0,0,0,0,0,0
4,"(PFK,)",8.581712,10.222978,0,0,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4090,"(GLCabcpp, GLCptspp, HEX1, PFK, FBA, TPI, GAPD...",0.000000,0.100671,1,1,1,0,1,1,1,1,1,1,1,1
4091,"(GLCabcpp, GLCptspp, PGI, PFK, FBA, TPI, GAPD,...",0.000000,0.813981,1,1,0,1,1,1,1,1,1,1,1,1
4092,"(GLCabcpp, HEX1, PGI, PFK, FBA, TPI, GAPD, PGK...",0.000000,0.720188,1,0,1,1,1,1,1,1,1,1,1,1
4093,"(GLCptspp, HEX1, PGI, PFK, FBA, TPI, GAPD, PGK...",0.000000,1.095238,0,1,1,1,1,1,1,1,1,1,1,1


In [66]:
df.to_csv("Ecoli_Suc_KO_12_PCA.csv")